In [2]:
import pandas as pd
import numpy as np
import sklearn
from scipy.stats.stats import pearsonr

In [3]:
'''Notes predict target. 1 means borrower had issues making payment 0 all other predict if borrower
will have problems paying back the loan. Visual data scan:
def_60_cnt_social bureau (loan payment problems in area)
Step 1 Find correcation between target and other variables
'''

'Notes predict target. 1 means borrower had issues making payment 0 all other predict if borrower\nwill have problems paying back the loan. Visual data scan:\ndef_60_cnt_social bureau (loan payment problems in area)\nStep 1 Find correcation between target and other variables\n'

In [4]:
df = pd.read_csv('../data/application_train.csv')
# df['DEF_30_CNT_SOCIAL_CIRCLE']
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# TODO recursive way to clean up data
def normalize_boolean(value):
    if value == type(bool):
        return value * 1

def replace_yes_no(value):
    if value == 'Y':
        return 1
    elif value == 'N':
        return 0

In [6]:
# Visual check for seemingly useless column lets drop
drop = [col for col in df if col.startswith('FLAG_DOCUMENT_')]
df = df.drop(drop, axis=1)
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,2.0,2.0,2.0,-1134.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,1.0,0.0,-828.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,-815.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,2.0,0.0,-617.0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,-1106.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Manual clean up bad! Keep for reference
df['FLAG_OWN_REALTY'] = df['FLAG_OWN_REALTY'].map({'Y':1, 'N':0})
df['FLAG_OWN_CAR'] = df['FLAG_OWN_CAR'].map({'Y':1, 'N':0})
df['CODE_GENDER'] = df['CODE_GENDER'].map({'M':1, 'F':0})
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,1.0,0,1,0,202500.0,406597.5,24700.5,...,2.0,2.0,2.0,-1134.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,0.0,0,0,0,270000.0,1293502.5,35698.5,...,0.0,1.0,0.0,-828.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,1.0,1,1,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,-815.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,0.0,0,1,0,135000.0,312682.5,29686.5,...,0.0,2.0,0.0,-617.0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,1.0,0,1,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,-1106.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# check for non numerical values
#df.select_dtypes(exclude=[np.number])
#check for dtypes in table
#g = df.columns.to_series().groupby(df.dtypes).groups

df_build_in_cat = pd.get_dummies(df)
df_build_in_cat.head()
#df_test = df_build_in_cat.fillna(value=-999999, inplace=True)

# try build in catergorizing of data for 1 column
# df['NAME_CONTRACT_TYPE'].astype('category')
# dtype(df['NAME_CONTRACT_TYPE'])

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1,1.0,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,0.0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,0,1,0,0,0,0,0,0,1,0
2,100004,0,1.0,1,1,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,0.0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,1.0,0,1,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Check for machine unfriendly values
df_build_in_cat.select_dtypes(exclude=[np.number])
print(df_build_in_cat.isnull().values.any())
df_build_in_cat.fillna(value=-999999, inplace=True)

if df_build_in_cat.select_dtypes(include=[object]).empty:
    print('No NaNs')

df_build_in_cat.head()

True
No NaNs


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1,1.0,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,0.0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,0,1,0,0,0,0,0,0,1,0
2,100004,0,1.0,1,1,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,0.0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,1.0,0,1,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# X features, y as label
# y = df_build_in_cat.target
# X = np.array(df_build_in_cat.drop(['TARGET', 'SK_ID_CURR'], axis=1), dtype='f')
# y what are we predicting
y = np.array(df_build_in_cat['TARGET'])
# X what are we using to predict
X =  np.array(df_build_in_cat.drop(['TARGET', 'SK_ID_CURR'], axis=1), dtype='f')

In [11]:
# Check np array for evil value
print('Do we have any NaN\'s: {}'.format(np.any(np.isnan(X))))
print('Make sure we have no infinity? {}'.format(np.all(np.isfinite(X))))

Do we have any NaN's: False
Make sure we have no infinity? True


In [12]:
from sklearn.tree import DecisionTreeRegressor
tree_regressor_model = DecisionTreeRegressor()

tree_regressor_model.fit(X, y)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [13]:
tree_regressor_model.predict(X)

array([1., 0., 0., ..., 0., 1., 0.])

In [14]:
from sklearn.metrics import mean_absolute_error
predictions = tree_regressor_model.predict(X)
mean_absolute_error(y, predictions)

0.0

In [15]:
'''
Merge them tables
'''
df_b_app =pd.merge(df_bureau, df, on='SK_ID_CURR')

NameError: name 'df_bureau' is not defined

In [50]:
'''
Normalize Bureau datas TODO
'''


In [10]:
'''
Check for correlation
'''
df[(df.TARGET == 1)][['TARGET', 'FLAG_OWN_REALTY']]
df[(df.TARGET == 1)][['TARGET','DEF_30_CNT_SOCIAL_CIRCLE']]
drop_na_df = df.dropna()
print(pearsonr(drop_na_df.TARGET, drop_na_df.DEF_30_CNT_SOCIAL_CIRCLE))

(0.02280714646369726, 0.03440767560293876)
